# Primeira versão do modelo

moeda escolhida: Bitcoin

Com base nas análises iniciais, o Bitcoin ofi o que tinha a tendência mais acentuada de crescimento.

Escolha do modelo: LSTM 
    Após pesquisas sobre modelos envolvendo previsão de cripto, a LSTM foi a mais indicada para séries temporais.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
btc_data = yf.download('BTC-USD', period='1y')

In [ ]:
data = btc_data['Close'] 
data = data.values.reshape(-1, 1) 

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

scaled_data = scaler.fit_transform(data)


In [ ]:
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

In [ ]:
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :1]

In [ ]:
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [4]:
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

C:\Users\Inteli\Inteli 2024\M7\m7-activities\venv\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
model.fit(X_train, y_train, epochs=epochs, batch_size=32)

Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 19s 143ms/step - loss: 0.0751
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - loss: 0.0084
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step - loss: 0.0050
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step - loss: 0.0043
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 135ms/step - loss: 0.0042
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 139ms/step - loss: 0.0033
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - loss: 0.0029
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 144ms/step - loss: 0.0036
Epoch 9/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - loss: 0.0029
Epoch 10/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - loss: 0.0027
Epoch 11/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - loss: 0.0029
Epoch 12/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 148ms/step - loss: 0.0037
Epoch 13/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 150ms/step - loss: 0.0026
Epoch 14/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 149ms/step - loss: 0.0025
Epoch 15/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s

In [ ]:
test_predict = model.predict(X_test)
test_predict = scaler.inverse_transform(test_predict)

In [ ]:
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
mean_squared_error(y_test_rescaled, test_predict)

In [ ]:
np.sqrt(mse)

In [ ]:
mean_absolute_error(y_test_rescaled, test_predict)

In [ ]:
# Pegando os últimos 'time_step' dias para fazer a previsão
last_data = scaled_data[-60:]
predictions = []
days_to_predict = 30

In [ ]:
for _ in range(30):
    last_data_reshaped = last_data.reshape(1, time_step, 1)
    next_pred = model.predict(last_data_reshaped)
    predictions.append(next_pred[0, 0])
    last_data = np.append(last_data[1:], next_pred, axis=0)

In [ ]:
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(predictions, color='red', label='previsão de preço do Bitcoin')
plt.plot(data, color='blue', label='Preço real do Bitcoin')
plt.title('Previsão de preço do Bitcoin para os próximos 30 dias')
plt.xlabel('dias')
plt.ylabel('preço em dólares')
plt.legend()
plt.show()

Apesar de não ter uma prvisão muito precisa, a LSTM é um modelo que pode ser ajustado para melhorar a previsão com mais épocas de treinamento e uma quantidade maior de dados.

Porém, pensando na implementação em uma aplicação web, foi decidido focar no funcionamento e estrutura da aplicação e possíveis melhorias no desempenho poderão ser feitas posteriormente.